In [ ]:
import pandas as pd
import numpy as np
import finterstellar as fs

# 데이터 준비하기

1. 시세데이터 불러오기

In [ ]:
file = 'w2-12 Back test practice data.xlsx'

In [ ]:
df = pd.read_excel(file, index_col='Date')
df

2. 데이터 전처리 : 날짜처리

In [ ]:
ld = fs.LoadData()
df = ld.date_formatting(df)    
# date_formatting(데이터프레임) - 날짜 처리
df

3. 파라미터 세팅 : 기준일자, 종목코드 등

In [ ]:
base_date = '2019-01-01'    # 기준일자 설정
cd = 'Inverse'    # 종목코드 

In [ ]:
v = fs.Visualize()
v.price_view(df, base_date, cd, (10,3))
# price_view(데이터프레임, 기준일자, 종목코드, (사이즈)) - 그래프 그리기

# 백테스팅 기능 구현 및 실행

트레이딩 로직의 기본 개념

In [ ]:
for i in df.index:    # 데이터프레임을 일자별로 순환
    price = df.loc[i, cd]    # 원하는 종목의 가격 읽어오기
    if price < 1000:    # 매수조건
        print(i, 'buy')
    elif price > 1000:    # 매도조건
        print(i, 'sell')

트레이딩 북 생성

In [ ]:
book = pd.DataFrame()
book[cd] = df[cd]
book['t '+cd] = ''
book['p '+cd] = ''
book

In [ ]:
def create_trade_book(sample, cd):
    book = pd.DataFrame()
    book[cd] = sample[cd]
    book['t '+cd] = ''
    book['p '+cd] = ''
    return(book)

In [ ]:
book = create_trade_book(df, cd)
book

In [ ]:
trd = fs.Trade()

In [ ]:
trd.create_trade_book(df, cd)
# create_trade_book(데이터프레임, 종목코드) - 시세판을 읽어 트레이드북 생성

트레이딩 결과 산출 및 트레이딩북에 기록 

In [ ]:
def trade(df, book, cd, buy, sell):
    for i in df.index:
        price = df.loc[i, cd]
        if price < buy:    # 가격이 매수기준 미만이면
            book.loc[i, 't '+cd] = 'buy'    # 매수 기록
        elif price > sell:    # 가격이 매도기준 초과이면
            book.loc[i, 't '+cd] = ''    # 포지션 청산 기록
        else:    # 가격이 매도, 매수구간 사이라면
            if book.shift(1).loc[i, 't '+cd] == 'buy':    # 이미 매수상태이면
                book.loc[i, 't '+cd] = 'buy'    # 매수상태 유지 기록

In [ ]:
trade(df, book, cd, 1000, 1000)
book

In [ ]:
trd.position(book, cd)
# posion(트레이드북, 종목코드) - 포지션 산출

수익률 산출 기능 구현 및 실행

In [ ]:
buy, sell, rtn = 0, 0, 1.0
for i in book.index:
    if book.loc[i, 'p '+cd] == 'zl':    # 매수 진입 시
        buy = book.loc[i, cd]    # 매수가 기록
        print('매수 @', buy)
    elif book.loc[i, 'p '+cd] == 'lz':    # 매도 진입 시
        sell = book.loc[i, cd]    # 매도가 기록
        earning = sell - buy    # 손익 계산
        earning_ratio = earning / buy * 100    # 수익률 계산
        print('매도 @', sell, '| 손익:', earning, '| 수익률:', round(earning_ratio, 1), '%')

In [ ]:
buy, sell = 0, 0
rtn = 1.0
book['return'] = 1
for i in book.index:
    if book.loc[i, 'p '+cd] == 'zl':    # 매수 진입 시
        buy = book.loc[i, cd]    # 매수가 기록
        print('매수 @', buy)
    elif book.loc[i, 'p '+cd] == 'lz':    # 매도 진입 시
        sell = book.loc[i, cd]    # 매도가 기록
        earning = sell - buy    # 손익 계산
        earning_ratio = earning / buy * 100    # 수익률 계산
        rtn = (sell - buy) / buy + 1    # 손익 계산
        print('매도 @', sell, '| 손익:', earning, '| 수익률:', round(earning_ratio, 1), '%')
        book.loc[i, 'return'] = round(rtn, 4)
        
acc_rtn = 1.0
for i in book.index:
    rtn = book.loc[i, 'return']
    acc_rtn = acc_rtn * rtn
    book.loc[i, 'acc return'] = acc_rtn

print ('Accunulated return :', round((acc_rtn - 1) * 100, 2), '%')

In [ ]:
book

In [ ]:
trd.returns(book, cd)
# returns(트레이드북, 종목코드) - 수익률 계산

In [ ]:
help(trd.returns)

In [ ]:
book

로그수익률 산출 기능 구현 및 실행

In [ ]:
buy, sell = 0, 0
for i in book.index:
    if book.loc[i, 'p '+cd] == 'zl':    # 매수 진입 시
        buy = book.loc[i, cd]    # 매수가 기록
        print('매수 @', buy)
    elif book.loc[i, 'p '+cd] == 'lz':    # 매도 진입 시
        sell = book.loc[i, cd]    # 매도가 기록
        earning = sell - buy    # 손익 계산
        earning_ratio = np.log(sell/buy) * 100    # 수익률 계산
        print('매도 @', sell, '| 손익:', earning, '| 수익률:', round(earning_ratio, 1), '%')

In [ ]:
buy, sell = 0, 0
rtn = 0.0
book['return'] = 0.0
for i in book.index:
    if book.loc[i, 'p '+cd] == 'zl':    # 매수 진입 시
        buy = book.loc[i, cd]    # 매수가 기록
        print('매수 @', buy)
    elif book.loc[i, 'p '+cd] == 'lz':    # 매도 진입 시
        sell = book.loc[i, cd]    # 매도가 기록
        earning = sell - buy    # 손익 계산
        earning_ratio = np.log(sell/buy) * 100    # 수익률 계산
        print('매도 @', sell, '| 손익:', earning, '| 수익률:', round(earning_ratio, 1), '%')
        book.loc[i, 'return'] = earning_ratio
        
acc_rtn = 0.0
for i in book.index:
    rtn = book.loc[i, 'return']
    acc_rtn = acc_rtn + rtn
    book.loc[i, 'acc return'] = acc_rtn
print ('Accunulated return :', round(acc_rtn, 2), '%')

In [ ]:
trd.returns_log(book, cd)
# returns(트레이드북, 종목코드) - 수익률 계산